## Import lib

In [ ]:
import os
path = '/content/drive/MyDrive/ProjectBigData/Data'
os.chdir(path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyvi

import numpy as np
from tqdm import tqdm

np.random.seed(42)
import pandas as pd
from pyvi import ViTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import BatchNormalization, MaxPooling1D, Conv1D#, Merge
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, LSTM, Bidirectional
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras import backend as K
from keras.models import model_from_json
from keras.models import load_model

In [ ]:
#download fasttext
#! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.vec.gz

In [ ]:
#extract fasttext
'''
import gzip
import shutil
with gzip.open('cc.vi.300.vec.gz', 'rb') as f_in:
    with open('cc.vi.300.vec', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        '''

## Import data

In [ ]:
########
df = pd.read_csv('/content/drive/MyDrive/ProjectBigData/Data/comb_extraSNS_ReINTEL.csv')
train, test = train_test_split(df, test_size=0.01, random_state=123)

In [ ]:
X_train = train["post_message"].values.astype(str)
y_train = train[['label']].values

## Function

In [ ]:
def preprocess(text):
    text = text.split(" ")
    return text

max_features =  30000
maxlen = 500
embed_size = 300
batch_size = 32
epochs = 5
l2_reg = .00001
filter_sizes = [3,4,5]
num_filters = 32
EMBEDDING_FILE = 'cc.vi.300.vec'

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
from keras.utils.data_utils import pad_sequences
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
x_train =pad_sequences(X_train, maxlen=maxlen)



embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embed_size))

max_features = num_words

for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Build Model

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

import tensorflow as tf

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=233)

checkpoint = ModelCheckpoint(path + '/Text-CNN/CNN-checkpoint.hdf5',
                             monitor='val_acc',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

def get_model():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Reshape((maxlen, embed_size, 1))(x)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), kernel_initializer='normal',
                    activation='elu')(x)

    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)

    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
    z = Flatten()(z)
    z = Dropout(0.35)(z)
    outp = Dense(1, activation="sigmoid")(z)



    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',f1_m,precision_m, recall_m])

    return model



In [ ]:
print(X_tra.shape, X_val.shape, y_tra.shape, y_val.shape)

(6025, 500) (1507, 500) (6025, 1) (1507, 1)


In [ ]:
model = get_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 500, 300)     5758200     ['input_2[0][0]']                
                                                                                                  
 spatial_dropout1d_1 (SpatialDr  (None, 500, 300)    0           ['embedding_1[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 reshape_1 (Reshape)            (None, 500, 300, 1)  0           ['spatial_dropout1d_1[0][0]

In [ ]:
history = model.fit(X_tra, y_tra, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/5
189/189 [==============================] - 233s 1s/step - loss: 0.4312 - acc: 0.8038 - f1_m: 0.5651 - precision_m: 0.7306 - recall_m: 0.4929 - val_loss: 0.3518 - val_acc: 0.8461 - val_f1_m: 0.7153 - val_precision_m: 0.8440 - val_recall_m: 0.6386
Epoch 2/5
189/189 [==============================] - 237s 1s/step - loss: 0.2943 - acc: 0.8823 - f1_m: 0.7656 - precision_m: 0.8616 - recall_m: 0.7091 - val_loss: 0.2777 - val_acc: 0.8912 - val_f1_m: 0.8019 - val_precision_m: 0.9114 - val_recall_m: 0.7289
Epoch 3/5
189/189 [==============================] - 236s 1s/step - loss: 0.2012 - acc: 0.9233 - f1_m: 0.8508 - precision_m: 0.9189 - recall_m: 0.8080 - val_loss: 0.2578 - val_acc: 0.8965 - val_f1_m: 0.8129 - val_precision_m: 0.9138 - val_recall_m: 0.7452
Epoch 4/5
189/189 [==============================] - 229s 1s/step - loss: 0.1356 - acc: 0.9530 - f1_m: 0.9139 - precision_m: 0.9534 - recall_m: 0.8884 - val_loss: 0.2527 - val_acc: 0.8998 - val_f1_m: 0.8240 - val_precision_m: 0.8883

In [ ]:
model.save(path + '/Text-CNN/CNN.h5')

## Load model

In [ ]:
load_model = load_model('/content/drive/MyDrive/ProjectBigData/Data/Text-CNN/CNN.h5',
                        custom_objects={"f1_m": f1_m, "precision_m": precision_m, "recall_m": recall_m})

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
predict = load_model.predict(X_val)

48/48 [==============================] - 6s 131ms/step


In [ ]:
rounded_predict = np.round(predict)
rounded_predict

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [1.]], dtype=float32)

In [ ]:
accuracy = accuracy_score(y_val, rounded_predict)
f1_score = f1_score(y_val, rounded_predict, average='macro')
roc_auc = roc_auc_score(y_val, rounded_predict)

# Print the evaluation metrics
print('Accuracy Score:', accuracy)
print('F1 Score:', f1_score)
print('ROC AUC Score:', roc_auc)

Accuracy Score: 0.9044459190444591
F1 Score: 0.8838177472823991
ROC AUC Score: 0.8682019159911569
